## Import Library

In [103]:
import imp
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options       # to customize chrome display
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC 
from time import sleep
from collections import Counter

import threading
import time
import pandas as pd
import numpy as np
from numpy import nan
import re
import concurrent.futures

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math

## Link search on e-commerce


In [104]:
# Link product search result
from turtle import delay
url = 'https://shopee.co.id/search?keyword=obat%20kanker'
path = '/Applications/chromedriver'

# create object for chrome options
chrome_options = Options()

# Customize chrome display
chrome_options.add_argument('start-maximized')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--headless')
chrome_options.add_argument('disable-notifications')

# To disable the message, "Chrome is being controlled by automated test software"
chrome_options.add_argument('--disable-infobars')   

# create webdriver object
driver = webdriver.Chrome(executable_path=path, options=chrome_options)
driver.get(url)



In [110]:
# Looping
rows = []

main_link = 'https://shopee.co.id/search?keyword=obat%20kanker&page={}'.format(page)
driver.get(main_link)
WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "shopee-search-item-result__items")))
driver.execute_script("""
var scroll = document.body.scrollHeight / 10;
var i = 0;
function scrollit(i) {
    window.scrollBy({top: scroll, left: 0, behavior: 'smooth'});
    i++;
    if (i < 10) {
    setTimeout(scrollit, 500, i);
    }
}
scrollit(i);
""")
sleep(5)
html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
soup = BeautifulSoup(html, "html.parser")

# Scrape product name
for item in soup.find_all('div', {'class': 'col-xs-2-4 shopee-search-item-result__item'}):
    product_name = item.find('div', {"ie3A+n bM+7UW Cve6sh"})
    if product_name is not None:
        product_name = product_name.text.strip()
    else:
        product_name = ''

# Scrape product price
    product_price = item.find('div', {'class':"vioxXd rVLWG6"})
    if product_price is not None:
        product_price = product_price.find('span', {'class': 'ZEgDH9'}).text.strip()
    else:
        product_price = ''

# Scrape total product sold
    product_sold = item.find('div', {'class':"r6HknA uEPGHT"})
    if product_sold is not None:
        product_sold = product_sold.text.strip()
    else:
        price = ''
    print([product_name, product_price, product_sold])
    rows.append([product_name, product_price, product_sold])



Page is ready
['Acep Herbal - Zedoril 7 Khusus Mengobati Tumor Benjolan Kanker Sipilis Raja Singa - BPOM & Halal', '255.000', '10RB+ Terjual']
['Obat Tumor Kista Bartholin Baker Ganglion Kanker Nasofaring Melanoma Darah Herbal De Nature', '315.000', '1,5RB Terjual']
['{Agen Resmi WISNUMART} Madu Bajakah Borneo ORI 100% Bergaransi - Atasi Benjolan , Kanker ,Tumor ,dan Segala Kronis', '165.000', '10RB+ Terjual']
['Obat Herbal Kanker| Kista| Tumor| Miom| Polip Hidung| Leukimia| De Nature Original| (Bisa COD)', '315.000', '10RB+ Terjual']
['IDR Madu Hitam  Obat Kanker Payudara  350 gram', '250.000', None]
['Flagelord Annamord Typogell Zirzax De Nature Capsule Obat Menghambat Dan Menyembuhkan Sel Kanker Tumor Ganas Herbal Ampuh Original', '175.000', '107 Terjual']
['Obat habutop,Habatop herbal isi 200 kapsul,untuk Stroke,Kanker,Migrain dan Hipertensi', '230.000', '51 Terjual']
['Cek Testimoni - Obat Kanker Serviks atau Rahim, Payudara, Prostat & Kelenjar Getah Bening', '60.000', '137 Terjua

In [114]:
rows[0]

['Acep Herbal - Zedoril 7 Khusus Mengobati Tumor Benjolan Kanker Sipilis Raja Singa - BPOM & Halal',
 '255.000',
 '10RB+ Terjual']

In [89]:
# Looping 1 pages
product_name, product_price, product_sold, product_city = [], [], [], []
for page in range(0,2):
    main_link = 'https://shopee.co.id/search?keyword=obat%20kanker&page={}'.format(page)
    driver.get(main_link)
    WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "shopee-search-item-result")))
    driver.execute_script("""
        var scroll = document.body.scrollHeight / 10;
        var i = 0;
        function scrollit(i) {
           window.scrollBy({top: scroll, left: 0, behavior: 'smooth'});
           i++;
           if (i < 10) {
            setTimeout(scrollit, 500, i);
            }
        }
        scrollit(i);
        """)
    sleep(5)
    html = driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
    soup = BeautifulSoup(html, "html.parser")

    # Scrape product name
    for i in soup.find_all('div', class_='ie3A+n bM+7UW Cve6sh'):
        product_name.append(i.text)

    # Scrape product price
    for i in soup.find_all('div', class_='hpDKMN'):
        product_price.append(i.text)

    # Scrape total product sold
    for i in soup.find_all('div', class_='r6HknA uEPGHT'):
        product_sold.append(i.text)
    
    #Scrape the city of the product sold
    for i in soup.find_all('div', class_='zGGwiv'):
        product_city.append(i.text)


In [90]:
len(product_name)

66

In [50]:
# Save data
listCols = ['product_name', 'product_price', 'product_sold']
dict_data = dict(zip(
    listCols,(
        product_name,
        product_price,
        product_sold)
))


In [51]:
# Save in DataFrame
df = pd.DataFrame(data=dict_data)

# The first five data
df.head()

,product_name,product_price,product_sold
0,"Obat Herbal Kanker Payudara, Serviks, Hati, Us...",Rp525.000,"3,8RB Terjual"
1,OBAT KANKER TUMOR MIOM KISTA KELENJAR POLIP L...,Rp200.000Rp184.000,"1,8RB Terjual"
2,OBAT KANKER TUMOR KELENJAR GETAH BENING LIMFOM...,Rp315.000,"8,1RB Terjual"
3,Walatra Zedoril 7 Asli Obat Herbal Kanker Tumo...,Rp255.000,10RB+ Terjual
4,GRAVIDA BHARATA OBAT KANKER PAYUDARA AMPUH |KA...,Rp275.000,10RB+ Terjual


In [52]:
#The last five data
df.tail()

,product_name,product_price,product_sold
760,Obat Benjolan Di Payudara Leher Obat Kelenjar ...,Rp140.000,151 Terjual
761,Obat Herbal Kanker Payudara Obat Kanker servik...,Rp70.000,"2,7RB Terjual"
762,Obat Kanker Tumor Kista Miom Kelenjar Tiroid G...,Rp200.000Rp184.000,"4,7RB Terjual"
763,Obat Kanker Payudara Serviks Tumor jinak ganas...,Rp70.000 - Rp250.000,748 Terjual
764,HEALTHYBEST Obat Kanker payudara penghancur be...,Rp195.000,290 Terjual


In [ ]:
#Export to CSV
df.to_csv('cancer-claim-drugs.csv', index=True)

In [ ]:
# close the automated browser
driver.close()